In [0]:
import spacy
import numpy as np
from pickle import dump,load
from google.colab import files

from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding

In [0]:
files.upload()  #upload your moby.txt file here

In [0]:
with open('moby.txt') as f:
  txt = f.read()

print("Text Length : ",len(txt))

nlp = spacy.load('en', disable = ['parser','tagger','ner'])
nlp.max_length = len(txt)*2

tokens = [token.text.lower() for token in nlp(txt) if token.text not in '\n\n \n\n\n!"-#$%&()-.*+,~/:;<=>?@[\\]^_`{|}\t\n ']

Text Length :  1232923


In [0]:
x = []
y = []

for i in range(0,len(tokens) -25 -1):
  x.append(tokens[i:i+25])
  y.append(tokens[i+25:i+26])

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)

x = tokenizer.texts_to_sequences(x)
y = tokenizer.texts_to_sequences(y)

x = np.array(x)
y = np.array(y)

y = to_categorical(y, num_classes = len(tokenizer.word_counts) + 1)

In [0]:
model = Sequential()

model.add(Embedding(len(tokenizer.word_counts)+1, 25, input_length = 25))
model.add(LSTM(50, return_sequences = True))
model.add(LSTM(50))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(len(tokenizer.word_counts)+1, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 25)            452225    
_________________________________________________________________
lstm_1 (LSTM)                (None, 25, 50)            15200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 18089)             922539    
Total params: 1,412,714
Trainable params: 1,412,714
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(x,y, batch_size = 256, epochs = 350, verbose = 1)

In [0]:
dct = list(tokenizer.word_index.keys())

for i in range(14,50):
  pred = np.reshape(x[i], (1,25))
  print(dct[model.predict_classes(pred)[0]], end = ' ')

a to a aye number as electronic is unharming was to included files fail keeping laws expatiate they check it these of water with to in employed of reported empire that hunters volition slaves encourage will 

In [0]:
tokenizer.word_index

In [0]:
model.save('300.h5')

In [0]:
dump(tokenizer,open('tokenizer','wb'))